# Transfer Learning

In [1]:
# Use another model trained in a different training set to distinguish other objects
# Can save a lot of time as one does not need to train the model again
# Just need to change the last (classification) layer of the model

In [2]:
# Image Folder
# Scheduler
# Transfer Learning

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [20]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [21]:
# import data
data_dir = 'data/hymenoptera_data'
sets = ['train', 'val']
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=0)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

['ants', 'bees']


In [27]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize only if in trainig phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        print()
        
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [28]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features   # number of input features for the last layer

model.fc = nn.Linear(num_ftrs, 2) # Assign a new layer to the last layer
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [29]:
# scheduler

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, 
                                        gamma=0.1)  # every 7 epochs, our learning rate is multiplied by gamma

#for epoch in range(100):
#    train() # optimizer.step()
#    evaluate()
#    scheduler.step()

In [ ]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.6587 Acc: 0.6475
val Loss: 0.4638 Acc: 0.7582

Epoch 1/19
----------
train Loss: 0.5932 Acc: 0.7131
val Loss: 0.3573 Acc: 0.8954

Epoch 2/19
----------
train Loss: 0.4831 Acc: 0.7459
val Loss: 0.2906 Acc: 0.8954

Epoch 3/19
----------
train Loss: 0.4383 Acc: 0.7828
val Loss: 0.2620 Acc: 0.9085

Epoch 4/19
----------
train Loss: 0.4137 Acc: 0.8525
val Loss: 0.2344 Acc: 0.9216

Epoch 5/19
----------
train Loss: 0.4510 Acc: 0.7951
val Loss: 0.2044 Acc: 0.9281

Epoch 6/19
----------
train Loss: 0.4584 Acc: 0.7705
val Loss: 0.1984 Acc: 0.9608

Epoch 7/19
----------
train Loss: 0.3168 Acc: 0.8811


In [ ]:
# Another option is to freeze the layers in front and only train the last layer
'''
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False   # Freezes all the layers in the beginning

num_ftrs = model.fc.in_features   # number of input features for the last layer

model.fc = nn.Linear(num_ftrs, 2) # Assign a new layer to the last layer
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mode.parameters(), lr=0.001)
'''